Accept the value of your portfolio and tell you how many shares of each S&P 500 constituent you should purchase to get an equal-weight version of the index fund.

In [4]:
!pip install xlsxwriter

     |████████████████████████████████| 149 kB 5.4 MB/s 


In [85]:
import pandas as pd
import numpy as np
import requests
import xlsxwriter
import math

Importing Stockss

In [67]:
stocks[stocks.Ticker == "WLTW"]

,Ticker


In [66]:
#importing saved list of S&P 500 stocks 
stocks = pd.read_csv("sp_500_stocks.csv")

In [8]:
#stocks
stocks.head()

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV


Acquiring an API Token

In [68]:
#setting API token
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

In [18]:
#Batch api calls
def chunks(lst, n):
  """
  z
  """
  for i in range(0, len(lst), n):
    yield lst[i:i+n]

In [94]:
symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#     print(symbol_strings[i])

final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'], 
                                                   data[symbol]['quote']['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)


In [95]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to buy
0,A,144.48,43815911872,N/A
1,AAL,17.11,10870671242,N/A
2,AAP,238.17,14738613727,N/A
3,AAPL,176.81,2830955758312,N/A
4,ABBV,143.73,254255674277,N/A
...,...,...,...,...
499,YUM,125.41,37562837496,N/A
500,ZBH,125.23,26332725732,N/A
501,ZBRA,509.02,27218344455,N/A
502,ZION,73.86,11241091513,N/A


In [40]:
#Caluculating number of shares to buy

In [101]:
portfolio_size = input("Enter Value of portfolio: ")

try:
  val = float(portfolio_size)
except ValueError:
  print("please enter integer, \n")
  portfolio_size = input("Enter Value of portfolio: ")
  val = float(portfolio_size)
  


Enter Value of portfolio: 1000000


In [102]:
position_size = val/len(final_dataframe.index)
for i in range(len(final_dataframe.index)):
  final_dataframe.loc[i, "Number of Shares to buy"] = math.floor(position_size/final_dataframe.loc[i, "Stock Price"])

In [103]:
final_dataframe

,Ticker,Stock Price,Market Capitalization,Number of Shares to buy
0,A,144.48,43815911872,13
1,AAL,17.11,10870671242,115
2,AAP,238.17,14738613727,8
3,AAPL,176.81,2830955758312,11
4,ABBV,143.73,254255674277,13
...,...,...,...,...
499,YUM,125.41,37562837496,15
500,ZBH,125.23,26332725732,15
501,ZBRA,509.02,27218344455,3
502,ZION,73.86,11241091513,26


Save file as XLSX file

In [104]:
#initialize 
writer = pd.ExcelWriter("reccomended trades.xlsx", engine = "xlsxwriter")
final_dataframe.to_excel(writer, "Recommened Trades", index = False)


In [105]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_format = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_format = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [107]:
column_formats = { 
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Market Capitalization', dollar_format],
                    'D': ['Number of Shares to Buy', integer_format]
                    }

for column in column_formats.keys():
    writer.sheets['Recommened Trades'].set_column(f'{column}:{column}', 20, column_formats[column][1])
    writer.sheets['Recommened Trades'].write(f'{column}1', column_formats[column][0], string_format)

In [108]:
writer.save()